In [1]:
import os
from flopy.discretization import StructuredGrid
import numpy as np
import flopy.mf6 as mf6

In [2]:
WS = os.mkdir("a_folder") 

In [3]:
model_name = 'B_pleasant'
nper, nlay, nrow, ncol,    delr, delc,   xoffset, yoffset, epsg, angrot = 1, 3, 60, 70,    40, 40,    554400, 389200, 3070, 0
modelgrid = StructuredGrid(delr=np.ones(ncol) * delr, delc=np.ones(nrow) * delc, xoff=xoffset, yoff=yoffset, angrot=angrot)

In [4]:
top = [{'filename': '../data/top.dat'}]          # 04 surfaces, 03 layers
botm = [{'filename': '../data/botm_000.dat'},
        {'filename': '../data/botm_001.dat'},
        {'filename': '../data/botm_002.dat'}]
k = [{'filename': '../data/k_000.dat'},          # KH
     {'filename': '../data/k_001.dat'},
     {'filename': '../data/k_002.dat'}]
k33 = [{'filename': '../data/k33_000.dat'},      # KV
       {'filename': '../data/k33_001.dat'},
       {'filename': '../data/k33_002.dat'}]
strt = [top[0]] * nlay                           # Top for starting heads
recharge = {
    0: {'filename': '../data/rch_000.dat'}}      # RCH    
irch = [{'filename': '../data/irch.dat'}]        # iRCH   local
spec_head_perimeter = {
    0: {'filename': '../data/chd_000.dat'}}      # chd

In [22]:
sim   = mf6.MFSimulation   (sim_name=model_name, version="mf6", exe_name="mf6", sim_ws = "WS")     
tdis  = mf6.ModflowTdis    (sim, time_units="days", nper=1, perioddata=[(1.0, 1, 1.0)])               
ims   = mf6.ModflowIms     (sim, complexity="moderate", outer_dvclose=0.001)                       
gwf   = mf6.ModflowGwf     (sim, modelname=model_name, save_flows=True)         
oc    = mf6.ModflowGwfoc   (gwf, head_filerecord=f'{gwf.name}.hds', budget_filerecord=f'{gwf.name}.cbc',saverecord=[('head', 'all'), ("budget", "all")])                                             
dis   = mf6.ModflowGwfdis  (gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=1)   
npf   = mf6.ModflowGwfnpf  (gwf, icelltype=1, k=k, k33=k33)                      
ic    = mf6.ModflowGwfic   (gwf, strt=strt)        
rch   = mf6.ModflowGwfrcha (gwf, recharge=recharge, irch=irch)

In [24]:
chd   = mf6.ModflowGwfchd  (gwf, stress_period_data=spec_head_perimeter)     

MFDataException: An error occurred in data element "botm" model "B_pleasant" package "chd_0". The error occurred while getting data in the "_get_data" method.
Additional Information:
(1) Unable to open file E:\15_REPOS\00_BETAMI\02_floPy__________________\04a_Pleasant_Lake\00_______________\WS\../data/botm_000.dat in mode r.  Make sure the file is not locked and the folder exists.
(2) Error occurred while adding dataset "stress_period_data" to block "period"

In [ ]:
chd   = mf6.ModflowGwfchd  (gwf, stress_period_data=spec_head_perimeter)     
lak   = mf6.ModflowGwflak  (gwf, boundnames=True, nlakes=1, connectiondata={'filename': '../data/lake_cn.dat'},
    packagedata=[[0, 290.85, 345, 'lake1']],perioddata={0: [[0, 'evaporation', 0.000715], [0, 'rainfall', 0.00209]]},surfdep=0.1)

In [ ]:
gwf.modelgrid.set_coord_info(xoff=xoffset, yoff=yoffset, crs=epsg)